In [20]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [21]:
# Import data

qb = pd.read_csv('../modeling/qb_value_modeling/qb_value_data/qb_with_values.csv')
qb = qb.drop(columns = ['Unnamed: 0'])
rb = pd.read_csv('../modeling/rush_offense_value_modeling/rush_offense_value_data/rb_with_values.csv')
rb = rb.drop(columns = ['Unnamed: 0'])
pass_def = pd.read_csv('../modeling/pass_defense_value_modeling/pass_defense_value_data/pass_defense_with_values.csv')
pass_def = pass_def.drop(columns = ['Unnamed: 0'])
rush_def = pd.read_csv('../modeling/rush_defense_value_modeling/rush_defense_value_data/rush_defense_with_values.csv')
rush_def = rush_def.drop(columns = ['Unnamed: 0'])

In [22]:
# Put data into one data frame that just has the game identifiers and value numbers

qb = qb.copy()[['season', 'week', 'team', 'opponent', 'score', 'opponent_score', 'qb', 'passing_value']]
rb = rb.copy()[['season', 'week', 'team', 'opponent', 'score', 'opponent_score', 'rushing_value', 'qb_rushing_value_pct']]
pass_def = pass_def.copy()[['season', 'week', 'team', 'opponent', 'score', 'opponent_score', 'pass_def_value']]
rush_def = rush_def.copy()[['season', 'week', 'team', 'opponent', 'score', 'opponent_score', 'rush_def_value']]
df = qb.copy()
df = df.merge(rb).merge(pass_def).merge(rush_def)
df = df.drop_duplicates()
df

,season,week,team,opponent,score,opponent_score,qb,passing_value,rushing_value,qb_rushing_value_pct,pass_def_value,rush_def_value
0,2014,1,ARI,LAC,18,17,C. Palmer,0.604567,0.194004,0.270270,0.493942,0.855757
1,2014,1,ATL,NO,37,34,M. Ryan,0.846419,0.721665,0.120968,0.254835,0.216091
2,2014,1,BAL,CIN,16,23,J. Flacco,0.310023,0.578230,0.073684,0.191127,0.693342
3,2014,1,BUF,CHI,23,20,E. Manuel,0.422696,0.763580,0.116162,0.696008,0.408553
4,2014,1,CAR,TB,20,14,D. Anderson,0.712989,0.312710,0.087719,0.843228,0.321939
...,...,...,...,...,...,...,...,...,...,...,...,...
4740,2022,10,TB,SEA,21,16,T. Brady,0.909411,0.611409,0.000000,0.372324,0.741143
4741,2022,10,TEN,DEN,17,10,R. Tannehill,0.570619,0.098195,0.187500,0.708770,0.570952
4742,2022,10,WAS,PHI,32,21,T. Heinicke,0.534081,0.675205,0.089172,0.897627,0.159860
4743,2022,11,GB,TEN,17,27,A. Rodgers,0.487381,0.269715,0.000000,0.043549,0.537460


In [25]:
# Fix team names

def fix_team_names(game, is_team=True):
    team_mapping = {
        'ARI':'Arizona Cardinals',
        'ATL':'Atlanta Falcons',
        'BAL':'Baltimore Ravens',
        'BUF':'Buffalo Bills',
        'CAR':'Carolina Panthers',
        'CHI':'Chicago Bears',
        'CIN':'Cincinnati Bengals',
        'CLE':'Cleveland Browns',
        'DAL':'Dallas Cowboys',
        'DEN':'Denver Broncos',
        'DET':'Detroit Lions',
        'GB':'Green Bay Packers',
        'HOU':'Houston Texans',
        'IND':'Indianapolis Colts',
        'JAX':'Jacksonville Jaguars',
        'KC':'Kansas City Chiefs',
        'OAK':'Las Vegas Raiders',
        'LV':'Las Vegas Raiders',
        'LAC':'Los Angeles Chargers',
        'LAR':'Los Angeles Rams',
        'LA':'Los Angeles Rams',
        'MIA':'Miami Dolphins',
        'MIN':'Minnesota Vikings',
        'NE':'New England Patriots',
        'NO':'New Orleans Saints',
        'NYG':'New York Giants',
        'NYJ':'New York Jets',
        'PHI':'Philadelphia Eagles',
        'PIT':'Pittsburgh Steelers',
        'SF':'San Francisco 49ers',
        'SEA':'Seattle Seahawks',
        'TB':'Tampa Bay Buccaneers',
        'TEN':'Tennessee Titans',
        'WSH':'Washington Football Team',
        'WAS':'Washington Football Team'
    }
    
    if is_team:
        return team_mapping[game['team']]
    
    else:
        return team_mapping[game['opponent']]
    
df['team_full'] = df.apply(lambda x: fix_team_names(x, is_team=True), axis=1)
df['opponent_full'] = df.apply(lambda x: fix_team_names(x, is_team=False), axis=1)
df.head()

,season,week,team,opponent,score,opponent_score,qb,passing_value,rushing_value,qb_rushing_value_pct,pass_def_value,rush_def_value,team_full,opponent_full
0,2014,1,ARI,LAC,18,17,C. Palmer,0.604567,0.194004,0.270270,0.493942,0.855757,Arizona Cardinals,Los Angeles Chargers
1,2014,1,ATL,NO,37,34,M. Ryan,0.846419,0.721665,0.120968,0.254835,0.216091,Atlanta Falcons,New Orleans Saints
2,2014,1,BAL,CIN,16,23,J. Flacco,0.310023,0.578230,0.073684,0.191127,0.693342,Baltimore Ravens,Cincinnati Bengals
3,2014,1,BUF,CHI,23,20,E. Manuel,0.422696,0.763580,0.116162,0.696008,0.408553,Buffalo Bills,Chicago Bears
4,2014,1,CAR,TB,20,14,D. Anderson,0.712989,0.312710,0.087719,0.843228,0.321939,Carolina Panthers,Tampa Bay Buccaneers


In [26]:
# Save raw data frame with values

df.to_csv('../data/value_models_combined.csv')

In [27]:
# Save data frame with past 6 games rolling stats

# Notes:
# Pure average for now, not weighted (possible future adjustment?)
# Resets every new season. Previous iterations had used the final 6 games of the last season to predict week 1,
# but not sure if thats the best way to do it since things vary a lot from season to season. Can revisit later.

offense_base = df.copy()[['season', 'week', 'team', 'qb']]
defense_base = df.copy()[['season', 'week', 'team']]

offense_rolling = df.groupby(by=['season', 'team', 'qb']).rolling(
    6, closed='left', min_periods=1).mean()[[
    'passing_value', 'rushing_value', 'qb_rushing_value_pct']].reset_index(
    level=['season', 'team',  'qb'])[['passing_value', 'rushing_value', 'qb_rushing_value_pct']]

defense_rolling = df[['season', 'week', 'team', 'pass_def_value', 'rush_def_value']].drop_duplicates().groupby(
    by=['season', 'team']).rolling(6, closed='left', min_periods=1).mean()[[
    'pass_def_value', 'rush_def_value']].reset_index(level=['season', 'team'])[['pass_def_value', 'rush_def_value']]

offense = offense_base.join(offense_rolling)
defense = defense_base.join(defense_rolling).dropna()

combined = offense.merge(defense, how='left')

<ipython-input-27-c84e907adb2a>:11: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['opponent', 'opponent_full', 'team_full'], dtype='object')
  offense_rolling = df.groupby(by=['season', 'team', 'qb']).rolling(


In [28]:
# NOTE: Pretty sure everything in previous code block is good, but do some checks next time i look at this

combined[combined.team=='LV'].tail(15)

,season,week,team,qb,passing_value,rushing_value,qb_rushing_value_pct,pass_def_value,rush_def_value
4297,2021,15,LV,D. Carr,0.290034,0.420381,0.143121,0.289654,0.452845
4331,2021,16,LV,D. Carr,0.313270,0.383868,0.147843,0.250183,0.482934
4365,2021,17,LV,D. Carr,0.309126,0.416879,0.098111,0.350940,0.569247
4398,2021,18,LV,D. Carr,0.395898,0.465191,0.129119,0.411653,0.525518
4418,2021,19,LV,D. Carr,0.328456,0.483899,0.125612,0.398477,0.555388
4455,2022,1,LV,D. Carr,NaN,NaN,NaN,NaN,NaN
4487,2022,2,LV,D. Carr,0.242689,0.510120,0.000000,0.095614,0.827463
4519,2022,3,LV,D. Carr,0.267996,0.504733,0.055556,0.406531,0.451468
4551,2022,4,LV,D. Carr,0.363132,0.560439,0.095456,0.301049,0.480408
4585,2022,5,LV,D. Carr,0.344785,0.618769,0.120429,0.370167,0.581035


In [29]:
df[df.team == 'LV'].tail(15)

,season,week,team,opponent,score,opponent_score,qb,passing_value,rushing_value,qb_rushing_value_pct,pass_def_value,rush_def_value,team_full,opponent_full
4297,2021,15,LV,CLE,16,14,D. Carr,0.368248,0.610630,0.070707,0.418501,0.608349,Las Vegas Raiders,Cleveland Browns
4331,2021,16,LV,DEN,17,13,D. Carr,0.187760,0.560582,0.054545,0.655399,0.925776,Las Vegas Raiders,Denver Broncos
4365,2021,17,LV,IND,23,20,D. Carr,0.631182,0.559281,0.186047,0.761303,0.198399,Las Vegas Raiders,Indianapolis Colts
4398,2021,18,LV,LAC,35,32,D. Carr,0.246262,0.594084,0.130682,0.230035,0.737191,Las Vegas Raiders,Los Angeles Chargers
4418,2021,19,LV,CIN,19,26,D. Carr,0.412575,0.832673,0.192308,0.186235,0.594935,Las Vegas Raiders,Cincinnati Bengals
4455,2022,1,LV,LAC,19,24,D. Carr,0.242689,0.510120,0.000000,0.095614,0.827463,Las Vegas Raiders,Los Angeles Chargers
4487,2022,2,LV,ARI,23,29,D. Carr,0.293302,0.499345,0.111111,0.717448,0.075474,Las Vegas Raiders,Arizona Cardinals
4519,2022,3,LV,TEN,22,24,D. Carr,0.553403,0.671853,0.175258,0.090085,0.538289,Las Vegas Raiders,Tennessee Titans
4551,2022,4,LV,DEN,32,23,D. Carr,0.289745,0.793759,0.195349,0.577519,0.882914,Las Vegas Raiders,Denver Broncos
4585,2022,5,LV,KC,29,30,D. Carr,0.681519,0.731097,0.019231,0.169965,0.391064,Las Vegas Raiders,Kansas City Chiefs


In [30]:
combined

,season,week,team,qb,passing_value,rushing_value,qb_rushing_value_pct,pass_def_value,rush_def_value
0,2014,1,ARI,C. Palmer,NaN,NaN,NaN,NaN,NaN
1,2014,1,ATL,M. Ryan,NaN,NaN,NaN,NaN,NaN
2,2014,1,BAL,J. Flacco,NaN,NaN,NaN,NaN,NaN
3,2014,1,BUF,E. Manuel,NaN,NaN,NaN,NaN,NaN
4,2014,1,CAR,D. Anderson,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
4740,2022,10,TB,T. Brady,0.535948,0.285129,0.007250,0.419632,0.358144
4741,2022,10,TEN,R. Tannehill,0.497345,0.403018,0.074553,0.557905,0.565941
4742,2022,10,WAS,T. Heinicke,0.304781,0.467361,0.150806,0.593203,0.614816
4743,2022,11,GB,A. Rodgers,0.362612,0.605432,0.077573,0.534240,0.326999


In [32]:
# fix team names

combined['team_full'] = combined.apply(lambda x: fix_team_names(x, is_team=True), axis=1)
# combined['opponent_full'] = combined.apply(lambda x: fix_team_names(x, is_team=False), axis=1)
combined.head()

,season,week,team,qb,passing_value,rushing_value,qb_rushing_value_pct,pass_def_value,rush_def_value,team_full
0,2014,1,ARI,C. Palmer,NaN,NaN,NaN,NaN,NaN,Arizona Cardinals
1,2014,1,ATL,M. Ryan,NaN,NaN,NaN,NaN,NaN,Atlanta Falcons
2,2014,1,BAL,J. Flacco,NaN,NaN,NaN,NaN,NaN,Baltimore Ravens
3,2014,1,BUF,E. Manuel,NaN,NaN,NaN,NaN,NaN,Buffalo Bills
4,2014,1,CAR,D. Anderson,NaN,NaN,NaN,NaN,NaN,Carolina Panthers


In [33]:
# Save aggregated data frame with values

combined.to_csv('../data/value_models_combined_6_game_rolling.csv')